In [28]:
import pandas as pd
import numpy as np
import geopandas as gpd
import salem
import json

In [29]:
Export = pd.read_csv("../1_Country_Emission/Export.csv")
# We want to make use of Gross_output

In [30]:
Export

,Country,Industries,TID,TFD,FID,FFD,HID,HFD,Gross_output,Tol_export
0,AFG,Agriculture,1.538278e+06,2.982180e+06,1.118980e+06,488129.032924,419297.800230,2.494051e+06,4.520458e+06,1.607109e+06
1,AFG,Fishing,8.205439e+04,1.021976e+05,3.289848e+04,694.220139,49155.910904,1.015034e+05,1.842520e+05,3.359270e+04
2,AFG,Mining and Quarrying,2.571404e+05,2.229562e+04,4.084204e+04,702.204809,216298.329572,2.159341e+04,2.794360e+05,4.154425e+04
3,AFG,Food & Beverages,4.822598e+05,8.496691e+05,6.116602e+03,416.376645,476143.215230,8.492527e+05,1.331929e+06,6.532978e+03
4,AFG,Textiles and Wearing Apparel,1.526769e+05,1.494909e+05,2.357707e+04,2232.773464,129099.781240,1.472581e+05,3.021677e+05,2.580985e+04
...,...,...,...,...,...,...,...,...,...,...
4910,ZWE,"Education, Health and Other Services",4.055454e+05,9.167422e+05,4.053208e+05,11057.979058,224.685887,9.056842e+05,1.322288e+06,4.163787e+05
4911,ZWE,Private Households,5.459519e+04,3.839821e+05,5.458529e+04,2463.884800,9.902411,3.815182e+05,4.385773e+05,5.704917e+04
4912,ZWE,Others,1.879805e+05,3.851717e+05,1.879705e+05,7405.652550,9.980098,3.777660e+05,5.731522e+05,1.953762e+05
4913,ZWE,Re-export & Re-import,2.629224e+05,4.401848e+05,2.629099e+05,14487.588642,12.486401,4.256972e+05,7.031072e+05,2.773975e+05


## Question: What should we do with ROW?
#### $$\text{concentration factors} = \frac{\text{concentration caused by sector i in country j}}{\text{total output of sector i in country j}}$$
#### We only have one total output including all sectors for ROW.
#### We split the total output of ROW to different sectors according to the each sector's fraction among all sectors of other 189 countries.
#### $$\text{AGR output of ROW} = \frac{\text{AGR output of 189 countries}}{\text{total output of 189 countries}} \times \text{total output of ROW}$$

### Get rid of ANT: Netherlands Antilles (Netherlands) and USR: USSR

## Update: instead of getting rid of these two countries, using concentrations factors for the rest of the world

In [31]:
# Export = Export.drop(index=range(120*26,121*26),axis=0)
# Export = Export.drop(index=range(174*26,175*26),axis=0)
# # Reset column names and row indexes
# Export = Export.reset_index(drop=True)

In [32]:
Countries = Export['Country'].unique().tolist()

In [33]:
'SUD' in Countries

True

In [34]:
sector_name = Export['Industries'].unique().tolist()[:-1]

In [35]:
# Assign sectors in EORA to sectors in CEDS
sectors = ['AGR/RCOO','RCOO','IND','IND','IND','IND','IND','IND','IND','IND','IND','IND','ENE',
           'IND','RCOC','RCOC','RCOC','RCOC','NRTR/SHP','0','0','RCOC/RCOO','RCOC','RCOR','0','0']
agr = [0]

ene = [i for i, x in enumerate(sectors) if x == 'ENE']

ind = [i for i, x in enumerate(sectors) if x == 'IND']

nrtr = [sectors.index('NRTR/SHP')]

rcor = [i for i, x in enumerate(sectors) if x == 'RCOR']

rcoc = [i for i, x in enumerate(sectors) if x == 'RCOC']
rcoc.append(sectors.index('RCOC/RCOO'))

rcoo = [i for i, x in enumerate(sectors) if x == 'RCOO']
rcoo.append(sectors.index('RCOC/RCOO'))
rcoo.append(sectors.index('AGR/RCOO'))

shp = [sectors.index('NRTR/SHP')]

#  What belongs to IND or RCO also belongs to SLV
slv = list(np.unique(ind + rcor + rcoc + rcoo))

# Every sector will have waste.
wst = list(np.unique(slv + shp + nrtr + ene + agr))

A = ['agr','ene','ind','nrtr','rcor','rcoc','rcoo','shp','slv','wst']

In [36]:
sector_pos = {}
sector_pos['agr'] = [0]
sector_pos['ene'] = [i for i, x in enumerate(sectors) if x == 'ENE']

sector_pos['ind'] = [i for i, x in enumerate(sectors) if x == 'IND']

sector_pos['nrtr'] = [sectors.index('NRTR/SHP')]

sector_pos['rcor'] = [i for i, x in enumerate(sectors) if x == 'RCOR']

sector_pos['rcoc'] = [i for i, x in enumerate(sectors) if x == 'RCOC']
sector_pos['rcoc'].append(sectors.index('RCOC/RCOO'))

sector_pos['rcoo'] = [i for i, x in enumerate(sectors) if x == 'RCOO']
sector_pos['rcoo'].append(sectors.index('RCOC/RCOO'))
sector_pos['rcoo'].append(sectors.index('AGR/RCOO'))

sector_pos['shp'] = [sectors.index('NRTR/SHP')]

#  What belongs to IND or RCO also belongs to SLV
sector_pos['slv'] = list(np.unique(ind + rcor + rcoc + rcoo))

# Every sector will have waste.
sector_pos['wst'] = list(np.unique(slv + shp + nrtr + ene + agr))

In [37]:
sector_pos

{'agr': [0],
 'ene': [12],
 'ind': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13],
 'nrtr': [18],
 'rcor': [23],
 'rcoc': [14, 15, 16, 17, 22, 21],
 'rcoo': [1, 21, 0],
 'shp': [18],
 'slv': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 21, 22, 23],
 'wst': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  21,
  22,
  23]}

In [38]:
list(np.array(sector_name)[shp])

['Transport']

In [39]:
total_output_sector = {}
for i in Countries[:-1]:
    total_output_sector[i]={}
    for j in A:
        total_output_sector[i][j]= {}
total_output_sector

{'AFG': {'agr': {},
  'ene': {},
  'ind': {},
  'nrtr': {},
  'rcor': {},
  'rcoc': {},
  'rcoo': {},
  'shp': {},
  'slv': {},
  'wst': {}},
 'ALB': {'agr': {},
  'ene': {},
  'ind': {},
  'nrtr': {},
  'rcor': {},
  'rcoc': {},
  'rcoo': {},
  'shp': {},
  'slv': {},
  'wst': {}},
 'DZA': {'agr': {},
  'ene': {},
  'ind': {},
  'nrtr': {},
  'rcor': {},
  'rcoc': {},
  'rcoo': {},
  'shp': {},
  'slv': {},
  'wst': {}},
 'AND': {'agr': {},
  'ene': {},
  'ind': {},
  'nrtr': {},
  'rcor': {},
  'rcoc': {},
  'rcoo': {},
  'shp': {},
  'slv': {},
  'wst': {}},
 'AGO': {'agr': {},
  'ene': {},
  'ind': {},
  'nrtr': {},
  'rcor': {},
  'rcoc': {},
  'rcoo': {},
  'shp': {},
  'slv': {},
  'wst': {}},
 'ATG': {'agr': {},
  'ene': {},
  'ind': {},
  'nrtr': {},
  'rcor': {},
  'rcoc': {},
  'rcoo': {},
  'shp': {},
  'slv': {},
  'wst': {}},
 'ARG': {'agr': {},
  'ene': {},
  'ind': {},
  'nrtr': {},
  'rcor': {},
  'rcoc': {},
  'rcoo': {},
  'shp': {},
  'slv': {},
  'wst': {}},
 'ARM'

In [40]:
for i in range(len(Countries)):
    if Countries[i] != "ROW":
        for j in A:
            temp_index = [index + 26*i for index in sector_pos[j]]
            total_output_sector[Countries[i]][j] = Export.iloc[temp_index]['Gross_output'].sum()
#     else:
#         for j in A:
#             total_output_sector[Countries[i]][j] = 0

In [2]:
total_output_sector

NameError: name 'total_output_sector' is not defined

In [42]:
total_output = {}
for i in Countries:
    total_output[i]={}

In [43]:
# Summation of the value insde a dictionary
def dictsum(myDict):      
    sum = 0
    for i in myDict: 
        sum = sum + myDict[i]     
    return sum

In [44]:
for i in Countries[:-1]:
    total_output[i] = dictsum(total_output_sector[i])
total_output['ROW'] = Export[Export['Country'] == 'ROW']['Gross_output'].sum()
total_output_sector['ROW'] = total_output['ROW']

In [54]:
total_output

{'AFG': 96310904.59478924,
 'ALB': 63679008.20845337,
 'DZA': 705945058.6614523,
 'AND': 14357005.432965346,
 'AGO': 470780540.20642114,
 'ATG': 6319458.930268912,
 'ARG': 2287707594.7065916,
 'ARM': 46342169.196642555,
 'ABW': 14273539.219098616,
 'AUS': 5575556130.90588,
 'AUT': 1976933899.5725527,
 'AZE': 200963945.31837723,
 'BHS': 43380838.11640766,
 'BHR': 179432287.84487027,
 'BGD': 966570915.3885646,
 'BRB': 21710268.116400786,
 'BLR': 87837426.04762125,
 'BEL': 2742173522.809593,
 'BLZ': 8400779.823492752,
 'BEN': 44748219.5330882,
 'BMU': 22211981.97281617,
 'BTN': 11658570.395277537,
 'BOL': 144452846.3778197,
 'BIH': 88174540.62556097,
 'BWA': 80290222.3363925,
 'BRA': 7657714182.074862,
 'VGB': 6318650.1736365985,
 'BRN': 60427343.56104044,
 'BGR': 364625596.46385473,
 'BFA': 55290336.22226781,
 'BDI': 12551350.773989247,
 'KHM': 102234278.43428738,
 'CMR': 141834694.9187048,
 'CAN': 7911740495.987911,
 'CPV': 8548019.54206621,
 'CYM': 19579535.36867847,
 'CAF': 8665444.73

In [46]:
file=open("total_output.json",'w')
json.dump(total_output,file)
file.close()

In [47]:
# for i in total_output_sector:
#     for j in total_output_sector[i]:
#         total_output_sector['ROW'][j] += total_output_sector[i][j]

In [48]:
# dominator = dictsum(total_output_sector['ROW'])
# for i in A:
#     total_output_sector['ROW'][i] = (total_output_sector['ROW'][i]/dominator) * total_output['ROW']

In [49]:
# # check correctness
# dictsum(total_output_sector['ROW']),total_output['ROW']

In [50]:
file=open("total_output_sector.json",'w')
json.dump(total_output_sector,file)
file.close()